# Challenge: Text Into Data

```yaml
Course:   DS 5001 
Module:   02 Text Models
Topic:    Text into Data Challenge
Author:   Ryan Lipps
Date:     14 October 2022 (revised)
```

## Purpose

Ww import a text using the  Clip, Chunk, and Split pattern.

Demonstrate how to tokenize a raw text and map an OHCO onto the resulting dataframe of tokens.

In this notebook, we use the pattern from `M02_01` on a new text.

## Recipe

### Create TOKEN table

1. Inspect source text, taking note of where it begins and ends and the header patterns.
2. Import the source text into a dataframe of line strings.
3. Extract the title.
4. Clip the cruft by using regexs for the beginning and end of the actual text.
5. Chunk by using a regex for chapter headings, assign lines, and group.
6. Split into paragraphs using new lines.
7. Split into sentences using regex.
8. Split into tokens using regex.

## Create VOBAB table

1. Get token value counts and save as data frame.

## Set Up

In [1]:
import pandas as pd

### Import Config

In [2]:
import configparser
config = configparser.ConfigParser()
config.read("../../../env.ini")
data_home = config['DEFAULT']['data_home']
output_dir = config['DEFAULT']['output_dir']

In [3]:
text_file = f"{data_home}/gutenberg/pg161.txt"
csv_file = f"{output_dir}/austen-sense-and-sensibility.csv" # The file we will create

In [4]:
OHCO = ['chap_num', 'para_num', 'sent_num', 'token_num']

## Import file into a dataframe

In [5]:
LINES = pd.DataFrame(open(text_file, 'r', encoding='utf-8-sig').readlines(), columns=['line_str'])
LINES.index.name = 'line_num'
LINES.line_str = LINES.line_str.str.replace(r'\n', '', regex=True).str.strip()

In [6]:
LINES.sample(20)

,line_str
line_num,
12132,him as I once used to think I might be with you;
3735,"Elinor was not inclined, after a little observ..."
12937,To learn more about the Project Gutenberg Lite...
487,"""I think you will like him,"" said Elinor, ""whe..."
10478,carriage stopt at the door--of her doubt--her ...
4304,assertion attended it. She turned towards Luc...
878,of the house. It had not been built many year...
241,hardly suppose I should neglect them. But as ...
11386,be happy. I know the summer will pass happily...


## Extract Title 

In [7]:
title = LINES.loc[0].line_str.replace('The Project Gutenberg EBook of ', '')
title

'Sense and Sensibility, by Jane Austen'

## Clip Cruft

In [8]:
clip_pats = [
    r"\*\*\*\s*START OF (?:THE|THIS) PROJECT",
    r"\*\*\*\s*END OF (?:THE|THIS) PROJECT"
]

In [9]:
pat_a = LINES.line_str.str.match(clip_pats[0])
pat_b = LINES.line_str.str.match(clip_pats[1])

In [10]:
line_a = LINES.loc[pat_a].index[0] + 1
line_b = LINES.loc[pat_b].index[0] - 1

In [11]:
LINES = LINES.loc[line_a:line_b]

In [12]:
LINES.head(30)

,line_str
line_num,
20,
21,
22,
23,
24,
25,
26,
27,
28,


In [13]:
LINES.tail(20)

,line_str
line_num,
12647,which strong family affection would naturally ...
12648,merits and the happiness of Elinor and Mariann...
12649,"as the least considerable, that though sisters..."
12650,"within sight of each other, they could live wi..."
12651,"between themselves, or producing coolness betw..."
12652,
12653,
12654,
12655,THE END


## Chunk by chapter

### Find all chapter headers

The regex will depend on the source text. You need to investigate the source text to figure this out.

In [14]:
chap_pat = r"^\s*(?:chapter|letter)\s+\d+"

In [15]:
chap_lines = LINES.line_str.str.match(chap_pat, case=False)

In [16]:
# Change to title case for formatting consistency
LINES.loc[chap_lines,'line_str'] = LINES.loc[chap_lines,'line_str'].apply(lambda x: x.title())

In [17]:
LINES.loc[chap_lines]

,line_str
line_num,
42,Chapter 1
196,Chapter 2
399,Chapter 3
561,Chapter 4
756,Chapter 5
858,Chapter 6
986,Chapter 7
1112,Chapter 8
1244,Chapter 9


### Assign numbers to chapters

In [18]:
LINES.loc[chap_lines, 'chap_num'] = [i+1 for i in range(LINES.loc[chap_lines].shape[0])]
LINES.loc[chap_lines]

,line_str,chap_num
line_num,,
42,Chapter 1,1.0
196,Chapter 2,2.0
399,Chapter 3,3.0
561,Chapter 4,4.0
756,Chapter 5,5.0
858,Chapter 6,6.0
986,Chapter 7,7.0
1112,Chapter 8,8.0
1244,Chapter 9,9.0


### Forward-fill chapter numbers to following text lines

`ffill()` will replace null values with the previous non-null value.

In [19]:
LINES.chap_num = LINES.chap_num.ffill()

### Clean up

In [20]:
LINES = LINES.dropna(subset=['chap_num'])      # Remove everything before chapter 1
LINES = LINES.loc[~chap_lines]                  # Remove chapter heading lines
LINES.chap_num = LINES.chap_num.astype('int')   # Convert chap_num to int

In [21]:
LINES.head()

,line_str,chap_num
line_num,,
43,,1
44,,1
45,The family of Dashwood had long been settled i...,1
46,"was large, and their residence was at Norland ...",1
47,"their property, where, for many generations, t...",1


### Group lines into chapters

In [22]:
OHCO[:1]

['chap_num']

In [23]:
# Change chapters into one big string
CHAPS = LINES.groupby(OHCO[:1])\
    .line_str.apply(lambda x: '\n'.join(x))\
    .to_frame('chap_str')

In [24]:
CHAPS.head(5)

,chap_str
chap_num,
1,\n\nThe family of Dashwood had long been settl...
2,\n\nMrs. John Dashwood now installed herself m...
3,\n\nMrs. Dashwood remained at Norland several ...
4,"\n\n""What a pity it is, Elinor,"" said Marianne..."
5,"\n\nNo sooner was her answer dispatched, than ..."


In [25]:
CHAPS['chap_str'] = CHAPS.chap_str.str.strip()

In [26]:
CHAPS.head()

,chap_str
chap_num,
1,The family of Dashwood had long been settled i...
2,Mrs. John Dashwood now installed herself mistr...
3,Mrs. Dashwood remained at Norland several mont...
4,"""What a pity it is, Elinor,"" said Marianne, ""t..."
5,"No sooner was her answer dispatched, than Mrs...."


## Split chapters into paragraphs 

We use Pandas' convenient `.split()` method with `expand=True`, followed by `.stack()`.
Note that this creates zero-based indexes.

In [27]:
para_pat = r'\n\n+'

In [28]:
PARAS = CHAPS['chap_str'].str.split(para_pat, expand=True).stack()\
    .to_frame('para_str').sort_index()
PARAS.index.names = OHCO[:2]

In [29]:
PARAS.head()

para_str
chap_num para_num                                                   
1        0         The family of Dashwood had long been settled i...
         1         By a former marriage, Mr. Henry Dashwood had o...
         2         The old gentleman died: his will was read, and...
         3         Mr. Dashwood's disappointment was, at first, s...
         4         His son was sent for as soon as his danger was...

In [30]:
PARAS['para_str'] = PARAS['para_str'].str.replace(r'\n', ' ', regex=True)
PARAS['para_str'] = PARAS['para_str'].str.strip()
PARAS = PARAS[~PARAS['para_str'].str.match(r'^\s*s')] # remove empty paragraphs

In [32]:
PARAS.head()

para_str
chap_num para_num                                                   
1        0         The family of Dashwood had long been settled i...
         1         By a former marriage, Mr. Henry Dashwood had o...
         2         The old gentleman died: his will was read, and...
         3         Mr. Dashwood's disappointment was, at first, s...
         4         His son was sent for as soon as his danger was...

## Split paragraphs into sentences

In [33]:
sent_pat = r'[.?!;:]+'
SENTS = PARAS['para_str'].str.split(sent_pat, expand=True).stack()\
    .to_frame('sent_str')
SENTS.index.names = OHCO[:3]

In [35]:
SENTS = SENTS[~SENTS['sent_str'].str.match(r'^\s*s')] # Remove empty paragraphs
SENTS.sent_str =  SENTS.sent_str.str.strip() # CRUCIAL TO REMOVE BLANK TOKENS
SENTS.head()

sent_str
chap_num para_num sent_num                                                   
1        0        0         The family of Dashwood had long been settled i...
                  1         Their estate was large, and their residence wa...
                  2         The late owner of this estate was a single man...
                  3         But her death, which happened ten years before...
                  4         for to supply her loss, he invited and receive...

## Split sentences into tokens

In [36]:
token_pat = r"[\s',-]+"
TOKENS = SENTS['sent_str'].str.split(token_pat, expand=True).stack()\
    .to_frame('token_str')

In [37]:
TOKENS.index.names = OHCO[:4]

In [38]:
TOKENS

token_str
chap_num para_num sent_num token_num             
1        0        0        0                  The
                           1               family
                           2                   of
                           3             Dashwood
                           4                  had
...                                           ...
50       22       0        8                  and
                           9          Sensibility
                           10                  by
                           11                Jane
                           12              Austen

[122727 rows x 1 columns]

## Extract Vocabulary

In [40]:
TOKENS['term_str'] = TOKENS.token_str.replace(r'[\W_]+', '', regex=True).str.lower()

In [41]:
VOCAB = TOKENS.term_str.value_counts().to_frame('n').reset_index().rename(columns={'index':'term_str'})
VOCAB.index.name = 'term_id'

In [42]:
VOCAB

,term_str,n
term_id,,
0,to,4084
1,the,4072
2,of,3553
3,and,3454
4,,2976
...,...,...
6252,score,1
6253,eternal,1
6254,station,1


## Gathering by Content Object

In [43]:
def gather(ohco_level):
    global TOKENS
    level_name = OHCO[ohco_level-1].split('_')[0]
    df = TOKENS.groupby(OHCO[:ohco_level])\
        .token_str.apply(lambda x: x.str.cat(sep=' '))\
        .to_frame(f'{level_name}_str')
    return df

In [44]:
gather(1)

,chap_str
chap_num,
1,The family of Dashwood had long been settled i...
2,Mrs John Dashwood now installed herself mistre...
3,Mrs Dashwood remained at Norland several month...
4,"""What a pity it is Elinor "" said Marianne ""tha..."
5,No sooner was her answer dispatched than Mrs D...
6,The first part of their journey was performed ...
7,Barton Park was about half a mile from the cot...
8,Mrs Jennings was a widow with an ample jointur...
9,The Dashwoods were now settled at Barton with ...


In [45]:
gather(2)

para_str
chap_num para_num                                                   
1        0         The family of Dashwood had long been settled i...
         1         By a former marriage Mr Henry Dashwood had one...
         2         The old gentleman died his will was read and l...
         3         Mr Dashwood s disappointment was at first seve...
         4         His son was sent for as soon as his danger was...
...                                                              ...
50       18        For Marianne however in spite of his incivilit...
         19        Mrs Dashwood was prudent enough to remain at t...
         20        Between Barton and Delaford there was that con...
         21                                                  THE END
         22        End of the Project Gutenberg EBook of Sense an...

[1811 rows x 1 columns]

In [46]:
gather(3)

sent_str
chap_num para_num sent_num                                                   
1        0        0         The family of Dashwood had long been settled i...
                  1         Their estate was large and their residence was...
                  2         The late owner of this estate was a single man...
                  3         But her death which happened ten years before ...
                  4         for to supply her loss he invited and received...
...                                                                       ...
50       20       0         Between Barton and Delaford there was that con...
                  1          and among the merits and the happiness of Eli...
                  2                                                          
         21       0                                                   THE END
         22       0         End of the Project Gutenberg EBook of Sense an...

[9313 rows x 1 columns]

In [48]:
sns_frame = gather(4)

In [49]:
sns_frame.head()

token_str
chap_num para_num sent_num token_num          
1        0        0        0               The
                           1            family
                           2                of
                           3          Dashwood
                           4               had

In [50]:
sns_frame['term_str'] = sns_frame['token_str'].str.lower()
sns_frame.head()

token_str  term_str
chap_num para_num sent_num token_num                    
1        0        0        0               The       the
                           1            family    family
                           2                of        of
                           3          Dashwood  dashwood
                           4               had       had

## Save work to CSV

This is important -- will be used for homework.